In [1]:
import brainlit
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from brainlit.algorithms.regression import *

import multiprocessing as mp
from multiprocessing import Pool, cpu_count
import os
import time
import threading
from functools import partial

import inspect

Using TensorFlow backend.


In [2]:
output_feats = "flat_neighbourhoods_3.csv" ##File will be changed to include larger dataset
df_iter = pd.read_csv(output_feats, header=None, index_col=0)
n_features = df_iter.shape[1] - 1
X_all = df_iter.iloc[:, :n_features]
y_all = df_iter.iloc[:, n_features:]

In [3]:
X_all = StandardScaler().fit_transform(X_all)
X_sel, X_test, y_sel, y_test = train_test_split(
    X_all, y_all, test_size=40, random_state=42)

In [4]:

def run_clf(X_train,y_train, X_test,y_test,runList_clf):
    clf  = runList_clf
    if(clf[1] == "MLP-relu-LR" ):
        trainStartTime = time.time()
        cls,his = MLP_LR_NN (X_train,y_train,124)
        trainEndTime = time.time()
        trainTime = trainEndTime - trainStartTime
        encoder = LabelEncoder()
        encoder.fit(y_test)
        en_y_test = encoder.transform(y_test)
                
        testStartTime = time.time()
        score = cls.evaluate(X_test, en_y_test, batch_size=20)
        testEndTime = time.time()
        testTime = testEndTime - testStartTime
        acc=score[1]
    else:
        #training
        trainStartTime = time.time()
        clf[0].fit(X_train, y_train)
        trainEndTime = time.time()
        trainTime = trainEndTime - trainStartTime
        #prediction
        testStartTime = time.time()
        out = clf[0].predict(X_test)
        testEndTime = time.time()
        testTime = testEndTime - testStartTime
        #accuracy
        acc = accuracy_score(y_test,out)
    #writing to file
    ####("variable,num of training samples,Lhat,avg precision,trainTime,testTime,iterate")
    #f.write(f"{clf[1]}, {n}, {acc:2.9f}, {trainTime:2.9f}, {testTime:2.9f}, {iteration}\n")
    #f.flush()
    return acc

In [6]:
names = {"MLP-LR": "black", "LR":"blue", "MLP-relu-LR":'red'}

ncores=1
num_runs=1
n_est=100
filename = 'multitry.csv'
classifiers = [
    MLPClassifier(hidden_layer_sizes=4, activation="logistic", alpha=1, max_iter=1000),
    LogisticRegression(max_iter=2000),
    MLP_LR_NN(X_sel, y_sel, n_features),
]


C:\Users\sanik\Anaconda3\envs\brainlight2\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sanik\Anaconda3\envs\brainlight2\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:
from multiprocessing import Process

if __name__ == '__main__':
    
    n = 10
    runList = [(clf) for clf in zip(classifiers, [key for key in names])]
    X_train = X_sel[:n]
    y_train = np.array(y_sel[:n]).ravel()

    
    p = Process(target=run_clf, args=(X_train,y_train,X_test,y_test,runList))
    p.start()
    p.join()

BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
#f = open(filename, 'w+')
#f.write("classifier,n,Accuracy,trainTime,testTime,iterate\n")
#f.flush()
pool = mp.Pool(processes = 3)
ns = np.array([10])
runList = [(clf) for clf in zip(classifiers, [key for key in names])]
st_time = time.time()
for n in tqdm(ns):
    print (n)
    for iteration in tqdm(range(num_runs)):
        X_train = X_sel[:n]
        y_train = np.array(y_sel[:n]).ravel()
        
        #X = X_train2[:n,:]
        #y = Y_train2[:n]
        print (iteration)
        func = partial(run_clf, X_train,y_train,X_test,y_test)
        accu = pool.map(func,runList)
        
        pool.close()
#f.close()
en_time = time.time()
tot_time = en_time - st_time
tot_time

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

10
0


In [ ]:
names = {"MLP-LR": "black", "LR": "blue", "MLP-relu-LR": "red"}
run_classifiers(
        X_sel, y_sel, X_test, y_test, classifiers, names, filename="test.csv")

In [2]:
from multiprocessing import Process

def f(name):
    print('hello', name)

if __name__ == '__main__':
    p = Process(target=f, args=('bob',))
    p.start()
    p.join()

In [ ]:
from multiprocessing import Pool

def f(x):
    return x*x

if __name__ == '__main__':
    with Pool(2) as p:
        print(p.map(f, [1, 2]))